# 5zju

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines)
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [11]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [12]:
nrun = 1000

1000

In [13]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1546]
        t2 = t[:, 1547:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [14]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1766.71,"[0.0594089, 0.981705, 0.180904]",148.686,"[34.6508, -26.0353, 129.906]",-3.99662,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 42.43 12.08 127.22 |"
2,1/complex.2.pdb,1719.1,"[-0.136829, -0.923018, 0.359605]",178.943,"[41.6442, 40.1767, 118.969]",0.866146,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 35.52 -0.54 116.31 |"
3,1/complex.3.pdb,1714.06,"[0.99842, -0.0481714, 0.0289296]",160.285,"[-4.37245, -15.8615, 124.491]",-27.0573,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 8.06 -52.76 113.89 |"
4,1/complex.4.pdb,1676.72,"[-0.504207, -0.126196, 0.854312]",118.631,"[61.8465, 10.3566, 38.0311]",-2.00556,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … -18.51 20.10 114.07 |"
5,1/complex.5.pdb,1667.03,"[-0.041054, -0.282934, -0.95826]",159.818,"[26.9256, -54.8819, 15.0507]",12.2761,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 37.68 -42.51 151.70 |"
6,1/complex.6.pdb,1666.78,"[-0.927668, -0.27046, -0.257457]",109.389,"[-25.9344, -26.3268, 121.103]",32.4591,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 19.62 -12.17 111.95 |"
7,1/complex.7.pdb,1657.03,"[0.293684, 0.35972, -0.885636]",179.906,"[68.4189, 26.5095, 33.4557]",-0.169676,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 31.48 -42.95 123.96 |"
8,1/complex.8.pdb,1656.61,"[-0.147622, -0.105607, 0.98339]",176.671,"[54.2034, 11.1395, 9.33306]",0.103954,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 29.11 -19.41 143.70 |"
9,1/complex.9.pdb,1628.79,"[0.848953, -0.0014553, 0.528466]",169.45,"[-52.6141, -23.9098, 84.4559]",-26.9138,"1x3092 TrjArray{Float64, Int64}\n| 34.00 4.16 126.81 | … 17.79 -61.28 128.50 |"


In [15]:
df_org = deepcopy(df);

In [16]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [17]:
#using BSON: @save, @load
#@save "df.bson" df

In [18]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)